In [9]:
import tensorflow as tf 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import cv2 
import random 
import datetime 
import time

In [10]:
styles = pd.read_csv('./fashion-dataset/styles.csv', error_bad_lines=False)
styles['images'] = styles.apply(lambda row: str(row['id']) + ".jpg", axis=1)
styles['images'] = styles['images'].apply(lambda x: os.path.join('./fashion-dataset/images/', x))
styles.head()

C:\Users\fancyma\AppData\Local\Temp\ipykernel_14356\2170906133.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  styles = pd.read_csv('./fashion-dataset/styles.csv', error_bad_lines=False)
Skipping line 6044: expected 10 fields, saw 11
Skipping line 6569: expected 10 fields, saw 11
Skipping line 7399: expected 10 fields, saw 11
Skipping line 7939: expected 10 fields, saw 11
Skipping line 9026: expected 10 fields, saw 11
Skipping line 10264: expected 10 fields, saw 11
Skipping line 10427: expected 10 fields, saw 11
Skipping line 10905: expected 10 fields, saw 11
Skipping line 11373: expected 10 fields, saw 11
Skipping line 11945: expected 10 fields, saw 11
Skipping line 14112: expected 10 fields, saw 11
Skipping line 14532: expected 10 fields, saw 11
Skipping line 15076: expected 10 fields, saw 12
Skipping line 29906: expected 10 fields, saw 11
Skipping line 31625: expected 10 fields, saw 11

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,images
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,./fashion-dataset/images/15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,./fashion-dataset/images/39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,./fashion-dataset/images/59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,./fashion-dataset/images/21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,./fashion-dataset/images/53759.jpg


In [11]:
# check unique labels
styles['masterCategory'].unique()

# check unique color
styles['baseColour'].unique().shape

(47,)

In [12]:
# read images and resize

def denoising(img):
    dst = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)
    return dst

# test contrast enhancement
def contrast_enhancement(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img[:,:,0] = cv2.equalizeHist(img[:,:,0])
    img = cv2.cvtColor(img, cv2.COLOR_YUV2RGB)
    return img

images_non = []
def load_data_images():
    images = []
    for i in range(0, 5000):
        img = cv2.imread(styles['images'][i]) # open cv thường đọc ảnh thành BGR nên cần convert sang RGB
        if img is None:
            images_non.append(i)
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # convert BGR to RGB
        img = denoising(img) # denoising image: loại bỏ nhiễu ảnh
        img = contrast_enhancement(img) # contrast enhancement: tăng độ tương phản
        img = cv2.resize(img, (224, 224))
        images.append(img)
    images = np.array(images)
    return images

images = load_data_images()

In [13]:
#  read labels and colors 
def load_data_labels():
    labels = []
    for i in range(0, 5000):
        if i in images_non:
            continue
        labels.append(styles['masterCategory'][i])
    labels = np.array(labels)
    return labels

labels = load_data_labels()

def load_data_colors():
    colors = []
    for i in range(0, 5000):
        if i in images_non:
            continue
        colors.append(styles['baseColour'][i])
    colors = np.array(colors)
    return colors

colors = load_data_colors()

In [14]:
# save data to pickle file
import pickle
pickle_out = open("./data_pickle/images.pickle", "wb")
pickle.dump(images, pickle_out)
pickle_out.close()

pickle_out = open("./data_pickle/labels.pickle", "wb")
pickle.dump(labels, pickle_out)
pickle_out.close()

pickle_out = open("./data_pickle/colors.pickle", "wb")
pickle.dump(colors, pickle_out)
pickle_out.close()


In [15]:
# load data from pickle file
import pickle
pickle_in = open("./data_pickle/images.pickle", "rb")
images = pickle.load(pickle_in)

pickle_in = open("./data_pickle/labels.pickle", "rb")
labels = pickle.load(pickle_in)

pickle_in = open("./data_pickle/colors.pickle", "rb")
colors = pickle.load(pickle_in)

# labels encoding 
from sklearn.preprocessing import LabelEncoder
le_labels = LabelEncoder()
labels = le_labels.fit_transform(labels)
print(le_labels.classes_)
print(le_labels.transform(le_labels.classes_))

# colors encoding
from sklearn.preprocessing import LabelEncoder
le_color = LabelEncoder()
colors = le_color.fit_transform(colors)
print(le_color.classes_)
print(le_color.transform(le_color.classes_))

['Accessories' 'Apparel' 'Footwear' 'Free Items' 'Personal Care']
[0 1 2 3 4]
['Beige' 'Black' 'Blue' 'Bronze' 'Brown' 'Burgundy' 'Charcoal'
 'Coffee Brown' 'Copper' 'Cream' 'Gold' 'Green' 'Grey' 'Grey Melange'
 'Khaki' 'Lavender' 'Lime Green' 'Magenta' 'Maroon' 'Mauve' 'Metallic'
 'Multi' 'Mushroom Brown' 'Mustard' 'Navy Blue' 'Nude' 'Off White' 'Olive'
 'Orange' 'Peach' 'Pink' 'Purple' 'Red' 'Rose' 'Rust' 'Sea Green' 'Silver'
 'Skin' 'Steel' 'Tan' 'Taupe' 'Teal' 'Turquoise Blue' 'White' 'Yellow']
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]


In [16]:
def define_model_2_labels(epochs=20, batch_size=32, validation_split=0.2):
    time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    inputs = tf.keras.layers.Input(shape=(224, 224, 3))
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    outputs = tf.keras.layers.Dense(7, activation='softmax')(x)
    outputs2 = tf.keras.layers.Dense(47, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=[outputs, outputs2])
    # plot_model
    tf.keras.utils.plot_model(model, to_file='./visualization_model/model.png', show_shapes=True,show_layer_names=True)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(images, [labels, colors], epochs=epochs, batch_size=batch_size, validation_split=validation_split)
    model.save(f'./models/model2label-{time}-{epochs}-{batch_size}-{validation_split}.h5')
    return model

In [17]:
def define_models():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128,activation='relu'))
    model.add(tf.keras.layers.Dense(7, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [18]:
model_2_labels = define_model_2_labels(20,64,0.2)

Epoch 1/20
63/63 [==============================] - 18s 108ms/step - loss: 33.4401 - dense_3_loss: 15.3430 - dense_4_loss: 18.0970 - dense_3_accuracy: 0.7990 - dense_4_accuracy: 0.1657 - val_loss: 3.1070 - val_dense_3_loss: 0.3194 - val_dense_4_loss: 2.7876 - val_dense_3_accuracy: 0.9130 - val_dense_4_accuracy: 0.2910
Epoch 2/20
63/63 [==============================] - 5s 87ms/step - loss: 2.7248 - dense_3_loss: 0.1887 - dense_4_loss: 2.5361 - dense_3_accuracy: 0.9513 - dense_4_accuracy: 0.3430 - val_loss: 2.7342 - val_dense_3_loss: 0.2459 - val_dense_4_loss: 2.4883 - val_dense_3_accuracy: 0.9270 - val_dense_4_accuracy: 0.3450
Epoch 3/20
63/63 [==============================] - 5s 87ms/step - loss: 2.1553 - dense_3_loss: 0.1217 - dense_4_loss: 2.0336 - dense_3_accuracy: 0.9688 - dense_4_accuracy: 0.4485 - val_loss: 2.7703 - val_dense_3_loss: 0.2469 - val_dense_4_loss: 2.5234 - val_dense_3_accuracy: 0.9410 - val_dense_4_accuracy: 0.3680
Epoch 4/20
63/63 [==============================] 

In [19]:
# load model
model_2_labels = tf.keras.models.load_model('./models/model_2_labels.h5')

# test model
def test_model_2_labels():
    img = cv2.imread('./fashion-dataset/images/15970.jpg')
    img = cv2.resize(img, (224, 224))
    img = np.array([img])
    pred = model_2_labels.predict(img)
    pred_label = np.argmax(pred[0])
    pred_color = np.argmax(pred[1])
    print('pred_label: ', le_labels.inverse_transform([pred_label]))
    print('pred_color: ', le_color.inverse_transform([pred_color]))
    plt.imshow(img[0])
    plt.show()
    # trust
    id = 15970
    df = pd.read_csv('./fashion-dataset/styles.csv', error_bad_lines=False)
    df = df[df['id'] == id]
    print(df['masterCategory'])
    print(df['baseColour'])

# test_model_2_labels()

# test model with all images
def test_model_2_labels_all():
    df = pd.read_csv('./fashion-dataset/styles.csv', error_bad_lines=False)
    predict_labels = []
    predict_colors = []
    trust_labels = []
    trust_colors = []
    for i in range(0, 10000):
        img = cv2.imread('./fashion-dataset/images/' + str(df['id'][i]) + '.jpg')
        if img is None:
            continue
        img = cv2.resize(img, (224, 224))
        img = np.array([img])
        pred = model_2_labels.predict(img)
        pred_label = np.argmax(pred[0])
        pred_color = np.argmax(pred[1])
        # print('pred_label: ', le_labels.inverse_transform([pred_label]))
        # print('pred_color: ', le_color.inverse_transform([pred_color]))
        predict_labels.append(le_labels.inverse_transform([pred_label])[0])
        predict_colors.append(le_color.inverse_transform([pred_color])[0])
        trust_labels.append(df['masterCategory'][i])
        trust_colors.append(df['baseColour'][i])
        # plt.imshow(img[0])
        # plt.show()
    return predict_labels, predict_colors, trust_labels, trust_colors
# compute accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, precision, recall, f1

predict_labels, predict_colors, trust_labels, trust_colors = test_model_2_labels_all()
accuracy, precision, recall, f1 = compute_metrics(trust_labels, predict_labels)
print('accuracy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)

C:\Users\fancyma\AppData\Local\Temp\ipykernel_14356\570468384.py:27: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('./fashion-dataset/styles.csv', error_bad_lines=False)
Skipping line 6044: expected 10 fields, saw 11
Skipping line 6569: expected 10 fields, saw 11
Skipping line 7399: expected 10 fields, saw 11
Skipping line 7939: expected 10 fields, saw 11
Skipping line 9026: expected 10 fields, saw 11
Skipping line 10264: expected 10 fields, saw 11
Skipping line 10427: expected 10 fields, saw 11
Skipping line 10905: expected 10 fields, saw 11
Skipping line 11373: expected 10 fields, saw 11
Skipping line 11945: expected 10 fields, saw 11
Skipping line 14112: expected 10 fields, saw 11
Skipping line 14532: expected 10 fields, saw 11
Skipping line 15076: expected 10 fields, saw 12
Skipping line 29906: expected 10 fields, saw 11
Skipping line 31625: expected 10 fields, saw 11
Ski

1/1 [==============================] - 0s 21ms/step


ValueError: y contains previously unseen labels: [5]

In [ ]:
accuracy, precision, recall, f1 = compute_metrics(trust_colors, predict_colors)
print('accuracy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)

accuracy:  0.870987098709871
precision:  0.8729863596501003
recall:  0.870987098709871
f1:  0.8707731119998425


In [ ]:
# test image in folder images_test

model = tf.keras.models.load_model('./models/model_2_labels.h5')

img = cv2.imread('./images_test/ao-thun-tron-mau-do-cho-nu.jpg')
img = cv2.resize(img, (224, 224))
img = np.array([img])
pred = model.predict(img)
pred_label = np.argmax(pred[0])
pred_color = np.argmax(pred[1])
print('pred_label: ', le_labels.inverse_transform([pred_label]))
print('pred_color: ', le_color.inverse_transform([pred_color]))

1/1 [==============================] - 7s 7s/step
pred_label:  ['Apparel']
pred_color:  ['Red']
